In [1]:
#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk
from nltk.tokenize import word_tokenize, RegexpTokenizer
import pandas as pd
from array import array
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.utils import shuffle
import csv
import numpy as np
from numpy import *
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import time
import h5py
from progressbar import ProgressBar
import fileinput
import pickle
import sklearn
from sklearn import metrics
from sklearn.cluster import SpectralClustering
import matplotlib.pyplot as plt
import datatable as dt
from sklearn.model_selection import train_test_split
from joblib import dump, load

In [2]:
filename = 'new_clean_all_data_with_title.csv'
# data = pd.read_csv(filename)

data = dt.fread(filename)
data = data.to_pandas()
data.head()

,link,sub_category,text,category,title
0,https://olahraga.kompas.com/read/2011/10/17/09...,Edukasi,Indonesian Corruption Watch menuntut Dinas Pe...,Edukasi,Icw Datangi Dinas Pendidikan Dki
1,https://travel.kompas.com/read/2009/05/14/2106...,Sains,Untuk mendapatkan populasi orangutan di Kalima...,Sains,Populasi Orangutan Disurvei
2,https://entertainment.kompas.com/read/2008/11/...,Sains,- Direktur Jenderal Sejarah dan Purbakala Dep...,Sains,Kota Majapahit Dibangun Dengan Sistem Jaringan...
3,https://entertainment.kompas.com/read/2008/09/...,Oase,"--Masyarakat Kristen Indonesia di Yogyakarta, ...",Wisata,Masyarakat Kristen Diy Temui Sultan
4,https://sains.kompas.com/read/2010/05/18/23143...,Liga Italia,"- Pelatih Inter Milan, Jose Mourinho, semakin ...",Olahraga,Mourinho Italia Tidak Menghargai Saya


In [5]:
# train, test = train_test_split(data, test_size=0.5, random_state=0, 
#                                stratify=data[['category']])

In [6]:
#sub category per main category
# cat_sub = []
# list_category = ['Ekonomi', 'Hiburan','Kesehatan', 'Lifestyle', 'Olahraga', 'Otomotif', 'Pendidikan', 'Properti', 'Sains', 'Teknologi', 'Wisata']
# for i in range(11):
#     cat_sub.append(data['sub_category'].loc[data['category']==list_category[i]].unique())
# cat_sub

In [7]:
#Articles per category
group = data.groupby('category')
print(group.size())
print("Shape :", data.shape)

category
Edukasi      2582
Ekonomi      2582
Hiburan      2582
Kesehatan    2582
Lifestyle    2582
Olahraga     2582
Otomotif     2582
Properti     2582
Sains        2582
Teknologi    2582
Wisata       2582
dtype: int64
Shape : (28402, 5)


In [ ]:
#PreProcessing

t0 = time.time()
# factory1 = StemmerFactory()
# stemmer = factory1.create_stemmer()
text = data['text']
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()
# stemming and stopword removal process
output = pd.DataFrame(columns = ['index','text'])
print ("Starting pre-precossing data (stop words removal)! (",text.shape[0], "articles)" )
pbar = ProgressBar()
i = 0
for x in pbar(text):
#     temp = stemmer.stem(text[i])
    temp = stopword.remove(x)
    temp = [[i,temp]]
    temp = pd.DataFrame(temp, columns = ['index','text'])
    output   = output.append([temp])
#     print('.', end='')
    i=+1
output = output.set_index('index')
print("Stop words removal is completed!")
print("===================================")
print("Starting pre-precossing data (punctuations removal)!")
# remove punctuations
tokenizer = RegexpTokenizer(r'\w+')
tagged_data = [TaggedDocument(words=tokenizer.tokenize(_d.lower()), tags=[str(c)]) for c, _d in enumerate(output.text)]
print ("Punctuations removal is completed!")
print("===================================")


t1= time.time()

print(len(output), "articles have been pre-processed!")

import pickle
with open('new_tagged_data.pkl', 'wb') as f:
    pickle.dump(tagged_data, f)

print("execution time: ", t1-t0)


In [8]:
#Load tagged_data
tagged_data1 = pickle.load( open( "new_tagged_data.pkl", "rb" ) )
len(tagged_data)

NameError: name 'tagged_data' is not defined

In [ ]:
#check whether there are empty data in tagged_data
empty_index = []
counter = len(tagged_data)
for i in range(counter):
    if not tagged_data[i][0]:
#         del tagged_data[i]
#         print ("index", str(i), "is deleted")
        empty_index.append(i)
len(empty_index)

In [21]:
#parameters for doc2vec model
max_epochs = 100
vec_size = 20
alpha = 0.025

In [ ]:
#Doc2Vec model
t0 = time.time()
model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)
print("Model Building")
for epoch in range(max_epochs):
    print("Epoch ", epoch+1)
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    

model.save("d2v.model")
print("execution time: ", time.time()-t0)

In [11]:
# #Load model
model= Doc2Vec.load("d2v.model")

In [24]:
docs = len(model.docvecs)
vocab = len(model.wv.vocab)
print("Documents Amount: ", docs)
print("Vocabulary: ", vocab)

Documents Amount:  28402
Vocabulary:  120382


In [25]:
#doc2vec metrics
npvec = np.zeros((docs,vec_size))

i = 0;
while model.docvecs[i] != "":
    npvec[i] = model.docvecs[i]
    i = i + 1
    if i == docs:
        break

/media/arif/Data1/ARIF/eureka/eurekeEnv/lib/python3.5/site-packages/ipykernel_launcher.py:5: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """


In [ ]:
# data_l1 = metrics.pairwise.cosine_similarity(npvec_try, npvec)
# print("%d bytes" % (y_kmeans.size * y_kmeans.itemsize))

In [ ]:
# npvec_try= []
# data_l1 = []
# del(data_l1)
# del(npvec_try)
# del(sc)

In [ ]:
#Extract data for NN
# np.savetxt("X.csv", npvec, delimiter=",")
# label = data['category']
# label.to_csv('y.csv')

In [ ]:
#Spectral CLustering
t0 = time.time()
npvec
data_l1 = metrics.pairwise.cosine_similarity(npvec)
label = 11
# sc = SpectralClustering(label,affinity='precomputed', n_init=100)
y_kmeans = sc.fit_predict(data_l1)
# color = ['red','yellow','blue']
#Visualising the clusters
for i in range(label):
    plt.scatter(npvec[y_kmeans == i, 0], npvec[y_kmeans == i, 1], s = 100, label = str(i+1))

plt.legend()
plt.show()
# dump(sc, 'sc_model.joblib')
print("Execution Time: ", time.time() - t0)

In [9]:
#load model
sc = load('sc_model.joblib')

/media/arif/Data1/ARIF/eureka/eurekeEnv/lib/python3.5/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.cluster.spectral module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.cluster. Anything that cannot be imported from sklearn.cluster is now part of the private API.
  warnings.warn(message, FutureWarning)
/media/arif/Data1/ARIF/eureka/eurekeEnv/lib/python3.5/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator SpectralClustering from version 0.19.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [12]:
#most common word based on category
list_category = ['ekonomi', 'hiburan','kesehatan', 'lifestyle', 'otomotif', 'pendidikan', 'properti', 'sains', 'teknologi', 'wisata', 'sepakbola']
category_count = len(list_category)
label_count = 0
label = 11
while label_count != label:
    lst = []
    sp = np.where(sc.labels_ == label_count)
    loopword = 0
#     while loopword != vocab:
#         word = model.wv.index2word[loopword]
    while loopword != category_count:
        word = list_category[loopword]
        wvec = model[word].reshape(1,-1)
        datatest = metrics.pairwise.cosine_similarity(model[sp], wvec)
        lst.append(datatest)
        loopword = loopword + 1
#     dfg = pd.DataFrame(lst).reshape(1,-1)
    file = "mostCommonWord_cluster" + str(label_count) + ".csv"
    lstnp =  np.asarray(lst)
    lstnp =  lstnp.reshape(category_count,lstnp.shape[1])
#     lstnp = np.abs(lstnp)
#     dfg.to_csv(file)
    np.savetxt(file, lstnp, delimiter=",")
    label_count = label_count + 1    

In [26]:
metrics.pairwise.cosine_similarity(model[sp], model['sepakbola'])

ValueError: Expected 2D array, got 1D array instead:
array=[-0.21205947  2.5159874   0.68523484 -1.8653946   0.3666308   1.4941964
  0.617962   -0.09356293 -0.09324379  0.4228653  -0.5036252  -0.04311667
  1.5967413   1.0011854  -0.32518235  1.162178   -1.0020175  -0.2915493
 -0.34907264  1.0368581 ].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [14]:
lstnp.shape

(11, 2237)

In [16]:
#Cluster label
cluster_label = []
common_words = []
for i in range(label):
    filename = "./mostCommonWord_cluster" + str(i) + ".csv"
    clusterdata = genfromtxt(filename, delimiter=',')
    cluster_mean = []
    temps = []
    for x in range(label):
        temp = [np.mean(clusterdata[x]),list_category[x]]
        temps.append(temp)
        cluster_mean.append(temp[0])
    cluster_label.append(list_category[np.argmax(cluster_mean)])
    common_words.append(temps)
# print('Cluster label : ', cluster_label)
for i in range (label):
    common_words[i].sort()

In [17]:
cluster_label

['kesehatan',
 'teknologi',
 'sepakbola',
 'lifestyle',
 'teknologi',
 'properti',
 'pendidikan',
 'lifestyle',
 'sains',
 'wisata',
 'properti']

In [18]:
pd.DataFrame(common_words[0])

,0,1
0,-0.072393,otomotif
1,-0.067781,sepakbola
2,-0.027262,wisata
3,-0.019472,properti
4,0.056399,lifestyle
5,0.111612,hiburan
6,0.117961,teknologi
7,0.171216,sains
8,0.215492,ekonomi
9,0.274463,pendidikan


In [27]:
pd.DataFrame(common_words[11])

IndexError: list index out of range

In [28]:
#vector all documents per cluster
header_row = ["Doc"]
for i in range (vec_size):
    temp = "vec" + str(i)
    header_row.append(temp)
for i in range (label):
    dump = []
    for x in range(docs):
        if sc.labels_[x] == i:
            dvec = model[x]
            doc = np.array([x])
            temp = np.concatenate((doc,dvec),axis=0)       
            dump = np.concatenate((dump,temp),axis=0)
    a = np.expand_dims(dump, axis=1)
    a = a.reshape(len(npvec[y_kmeans == i]), vec_size+1)
    file = "cluster" + str(i) + ".csv"
    with open(file, 'w', encoding="UTF-8", newline='') as myfile:
        wr = csv.writer(myfile)
#         wr.writerow(("Cluster ", label))
        wr.writerow((header_row))
        wr.writerows(a)
    myfile.close()

NameError: name 'y_kmeans' is not defined

In [ ]:
#Final result
predicted_label = []
cluster_no = []
for i in range (docs):
    for x in range (label):
        if sc.labels_[i] == x:
            predicted_label.append(cluster_label[x])
            cluster_no.append(x)

In [ ]:
#Top-5
top_5 = []
for i in range (docs):
    for x in range (label):
        if sc.labels_[i] == x:
            temp = []
            for j in range (6, 11):
                temp.append(common_words[x][j][1])
            top_5.append(temp)
#             top_5.append(common_words[x])

In [ ]:
final_data = pd.concat([data['link'],data['category'], pd.DataFrame(cluster_no, columns= ['cluster']), pd.DataFrame(predicted_label, columns= ['predicted_label'])], axis=1)
final_data.loc[final_data.category == 'Edukasi', 'category'] = 'Pendidikan'
final_data.loc[final_data.category == 'Olahraga', 'category'] = 'Sepakbola'
bestresult = np.zeros(docs)
for i in range (docs):
    if final_data.category[i].lower() == final_data.predicted_label[i]:
        bestresult[i]=1
final_data = pd.concat([final_data, pd.DataFrame(bestresult, columns=['score'])], axis=1)
top_5 = pd.concat([data['link'],data['category'], pd.DataFrame(cluster_no, columns= ['cluster']),pd.DataFrame(top_5, columns = ["label5","label4","label3","label2","label1"])], axis=1)
top_5.loc[top_5.category == 'Edukasi', 'category'] = 'Pendidikan'
top_5.loc[top_5.category == 'Olahraga', 'category'] = 'Sepakbola'
final_data.to_csv('final_data.csv')
top_5.to_csv('top_5.csv')

In [ ]:
#Best 5
top5label =  ["label5","label4","label3","label2","label1"]
top5result = np.zeros(docs)
for i in range (docs):
    for lab in top5label:
        if top_5.category[i].lower() == top_5[lab][i]:
            top5result[i]=1
finaltop5 = pd.concat([top_5['category'], top_5['cluster'], pd.DataFrame(top5result, columns=['score'])], axis=1)
finaltop5.to_csv('final_score_top5.csv')

In [23]:
#Best 3
top3label =  ["label3","label2","label1"]
top3result = np.zeros(docs)
for i in range (docs):
    for lab in top3label:
        if top_5.category[i].lower() == top_5[lab][i]:
            top3result[i]=1
finaltop3 = pd.concat([top_5['category'], top_5['cluster'], pd.DataFrame(top3result, columns=['score'])], axis=1)
finaltop3.to_csv('final_score_top3.csv')

NameError: name 'docs' is not defined

In [ ]:
print("Best 1 Accuracy: ", (final_data.loc[final_data.score==1].shape[0]/docs)*100, "%")
print("Best 3 Accuracy: ", (finaltop3.loc[finaltop3.score==1].shape[0]/docs)*100, "%")
print("Best 5 Accuracy: ", (finaltop5.loc[finaltop5.score==1].shape[0]/docs)*100, "%")

In [ ]:
for i in range(label):
    print("n_count cluster", i, " = ", len(np.where(sc.labels_ == i)[0]), "Cluster label: ", cluster_label[i])
print(list_category)

In [ ]:
final_result = pd.concat([final_data, finaltop3['score'], finaltop5['score']], axis=1)
final_result.columns = ['link', 'category', 'cluster', 'predicted_label', 'best1score', 'best3score',
       'best5score']
final_result.to_csv("final_result.csv")

In [ ]:
final_result